**----Challenge The Airbnb Database----------**

1.What's the most expensive listing? What else can you tell me about the listing?
~~~mysql
SELECT *
FROM sfo_listings
ORDER BY price DESC LIMIT 1
~~~
From the above query result we can see that the most expensive listing was 10000.To find out more about that listing,lets see what kind of review does it have.
~~~mysql
WITH
  most_expensive
AS(SELECT *
   FROM sfo_listings
   ORDER BY price DESC
   LIMIT 1
   )
SELECT m.*,s.review_date,s.comments
FROM
most_expensive m
JOIN
sfo_reviews s
ON
m.id = s.listing_id
~~~

2.What neighborhoods seem to be the most popular?
I have assumed that the availability of the listing on the calander table tells whether it is vacant(t) or accupied(f) but the listing is active.There is no clearity in the data if availability is based on what i have mentioned or the availability of that listing at all. 
~~~mysql
WITH
    occupied
AS(
   SELECT *
 	FROM
 	   sfo_calendar
	WHERE
	   available = 'f' 
  ),avai_listing
AS(SELECT id,neighbourhood,availability_365
   FROM 
   sfo_listings
   WHERE availability_365 != '0'
   )
   SELECT a.neighbourhood,COUNT(O.available) total_occupied
   FROM 
   occupied o
   JOIN
   avai_listing a
   ON
   o.listing_id = a.id
   GROUP BY a.neighbourhood
   ORDER BY total_occupied DESC LIMIT 3
  ~~~
  So the above sql code gives the three most occupied neighbourhoods in San Francisco;which from our deffinition in this case the most popular neighbourhoods in San Francisco.

**3.**<br/> 
**A.** In trying to find the cheapest time of year,i tried to fing out at which month of year is the average price the cheapest.So i grouped the data in to months and applied the average function after putting the data in appropriate format for computation.Finally i desplayed the month with the cheapest average monthly price and the price.The following is the SQL code for exectuting the above senario.
~~~mysql
WITH
      exclude_comma
AS(SELECT EXTRACT(MONTH FROM calender_date) AS months,REPLACE(price,',','') AS price
  FROM sfo_calendar
 WHERE price IS NOT null
   ),cast_float 
   AS(SELECT months,SUBSTR(price,2) AS price
	  FROM exclude_comma
	  )
SELECT months,AVG(CAST(price AS float)) AS price	
FROM cast_float
GROUP BY months
ORDER BY price
LIMIT 1

~~~
**B.** The busiest means more listings are occupied or less available,which is opposite to the above senario.
~~~mysql
WITH
  availability
AS(SELECT c.*,l.price As prices
FROM sfo_calendar c
JOIN
sfo_listings l
ON
c.listing_id = l.id
)
SELECT calender_date,
       COUNT(available) AS total_occupied
FROM availability
WHERE
available ='f'
GROUP BY calender_date
ORDER BY total_occupied
LIMIT 5
~~~